# Multi-Image Transformation

This example notebook demonstrates how to load and transform a series of RHEED images taken during sequential azimuthal rotations. 

The rotation starts at $\alpha = 0^\circ$, where the electron beam is aligned along the $[11\bar{2}]$ direction, and ends at $\alpha = 30^\circ$, corresponding to alignment along the $[1\bar{1}0]$ direction.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import xrheed
from xrheed.plotting.overview import plot_images
from xrheed.preparation.filters import high_pass_filter

## Image Preprocessing and Correction

To begin, we load a series of images and apply the necessary corrections.  
Due to minor manipulator inaccuracies, each image may exhibit slight variations in incident angle and/or positional shifts along the $S_x$ and $S_y$ axes.

Typically, these variations change linearly during the rotation.  

For instance, the incident angle can be interpolated as a linear space between the values recorded for the first and last images.  

The same approach applies to the azimuthal angle, α.


In [ ]:
image_dir = Path("example_data")
image_paths = list(sorted(image_dir.glob("Si_111_r3Ag_thB_phi*.raw")))

n_images = len(image_paths)

# Define alphas for each image (include manipulator inaccuracies)
alpha_vect = np.linspace(0.0, 29.7, n_images)
beta_vect = np.linspace(2.9, 3.5, n_images)

# Define center position for each image
center_x = -4.4
center_y_vect = np.linspace(-4.0, -6.7, n_images)

# Empty list to store rheed images
rheed_images = []

for idx, image_path in enumerate(image_paths):
    rheed_image = xrheed.load_data(image_path, plugin="dsnp_arpes_raw")

    # align each image
    rheed_image.ri.rotate(-0.2)
    rheed_image.ri.apply_image_center(center_x, center_y_vect[idx])
    rheed_image.ri.beta = beta_vect[idx]
    rheed_image.ri.alpha = alpha_vect[idx]

    rheed_image.ri.screen_roi_width = 60
    rheed_image.ri.screen_roi_height = 80

    # optionally use high pass filter
    rheed_image = high_pass_filter(rheed_image, sigma=2.0, threshold=0.5)

    rheed_images.append(rheed_image)


plot_images(rheed_images, ncols=2, vmin=3, vmax=25)
plt.show()

## Prepare a Lattice Object for the Expected Reconstruction

To verify the alignment, we create a lattice object representing the expected (√3 × √3) R30° reconstruction.  

This lattice will later be used by the `Ewald` object for spot calculation and overlay.


In [ ]:
from xrheed.kinematics.lattice import Lattice

# Si(111)-(1x1)
si_111_1x1 = Lattice.from_surface_hex(a=3.84, label="(1x1)")

# Si(111)-(r3xr3)R30
si_111_r3 = Lattice.from_surface_hex(a=3.84 * np.sqrt(3), label="(√3 x √3)")
si_111_r3.rotate(30)

## Generate Ewald Objects and Overlay Expected Spot Positions

We generate a series of `Ewald` objects and use them to overlay the expected spot positions on each image.  
This allows for visual verification of the alignment and consistency across the dataset.


In [ ]:
from xrheed.kinematics.ewald import Ewald

fig, axs = plt.subplots(n_images, 1, sharex=True, figsize=(4, 16))

for idx in range(n_images):
    ax = axs[idx]
    ew = Ewald(si_111_r3, rheed_images[idx])

    ew.plot(
        ax=ax,
        show_image=True,
        show_center_lines=False,
        auto_levels=0.2,
        marker="o",
        facecolor="none",
        edgecolor="c",
        s=80,
    )
    if idx < n_images - 1:
        ax.set_xlabel("")

plt.tight_layout()
plt.show()

## Transform All Images to kx–ky Space

If the alignment appears satisfactory, we proceed by transforming all images into the kx–ky space.  
This step uses the arguments `rotate=True` and `point_symmetry=True`.  

- The `rotate=True` option applies a rotation around the image center by the azimuthal angle α.  
- The `point_symmetry=True` option adds a mirrored image rotated by 180°, which is valid for most crystallographic structures.


In [ ]:
from xrheed.conversion.image import transform_image_to_kxky

trans_images = []
for rheed_image in rheed_images:
    trans_image = transform_image_to_kxky(rheed_image, rotate=True, point_symmetry=True)
    trans_images.append(trans_image)

## Create a Dataset

Convert the list of images into a `Dataset` object, introducing a new dimension that represents the azimuthal orientations.  
This structure facilitates organized access and further processing of the data across different angular configurations.


In [ ]:
import xarray as xr

# Create a dataset from the list of DataArrays
trans_image_dataset = xr.concat(trans_images, dim="alpha_vect")

# Assign the alpha_vect values to the new dimension
trans_image_dataset = trans_image_dataset.assign_coords(alpha_vect=alpha_vect)

## Compute the Mean Along the Azimuthal Dimension

Calculate the mean of the dataset along the azimuthal (α) dimension.  
This operation reduces the data to a single representative image, averaging over all angular orientations.


In [ ]:
trans_image_mean = trans_image_dataset.mean(dim="alpha_vect")

## Prepare the Final Plot

Finally, we generate an image that displays the mean intensity across all frames, along with overlays of the (1×1) and (√3 × √3) R30° reciprocal lattices for comparison.

As observed, the diffraction spots are elongated along several directions.  
This elongation is a characteristic feature of RHEED, resulting from the finite domain size, which affects the shape of the diffraction spots.

Nonetheless, in this representation, it is straightforward to identify specific surface reconstructions, their symmetries, and mutual orientations.


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

trans_image_mean.plot(ax=ax, cmap="gray", add_colorbar=False, vmin=5, vmax=15)

si_111_r3.plot_reciprocal(ax=ax, facecolor="none", edgecolor="c", s=150)
si_111_1x1.plot_reciprocal(ax=ax, facecolor="none", edgecolor="m", s=50)

ax.set_aspect(1)
ax.set_facecolor("black")
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
ax.legend(loc="upper right")

plt.show()